In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [211]:
NBC_df = pd.read_excel(r'C:\Users\bhavrang\Desktop\bhavya\Analytics\NBC_data_jan_aug.xlsx')

In [64]:
CURRENCY_DM_MAP_DF=pd.read_excel(r'C:\Users\bhavrang\Desktop\bhavya\Analytics\DM_ID_MAPPING.xlsx',sheet_name='currency_mapping')
DEVICE_DM_MAP_DF=pd.read_excel(r'C:\Users\bhavrang\Desktop\bhavya\Analytics\DM_ID_MAPPING.xlsx',sheet_name='device_mapping')

In [ ]:
#DEVICE_DM_MAP_DF.dtypes
DEVICE_DM_MAP_DF['DEVICE_DM_ID']=DEVICE_DM_MAP_DF['DEVICE_DM_ID'].astype(str)

In [174]:
#CURRENCY_DM_MAP_DF.dtypes
CURRENCY_DM_MAP_DF['CURRENCY_DM_ID']=CURRENCY_DM_MAP_DF['CURRENCY_DM_ID'].astype(str)

In [215]:
df=NBC_df
# to drop all 'DT_DM_ID' columns
li=[]
for i in df:
    if ('DT_DM_ID' in i):
        li.append(i)
df.drop(df[li],axis =1,inplace=True)

In [216]:
#to split summary description column to 2 new fields as subscription start date and end date. 
#two more fields are added to dataframe.
summary=df['SUMMARY_DESC'].str.split('for',expand=True)
timedf=summary[1].str.split('-',expand=True)
df['SUB_ST_DT']=timedf[0]
df['SUB_END_DT']=timedf[1]
df['NEW_SUMMARY_DESC']=summary[0]

In [217]:
#to convert invalid promocode values to 'INVALID'
df['PROMO_CODE']=df['PROMO_CODE'].apply(lambda x: 'INVALID' if (type(x)!=str) else x)

In [218]:
list_str_op=['AUTO_RENEW_STATUS','DATA_SOURCE_STATUS','DEL_IND','PAYMENT_SOURCE_TYPE','CREATED_BY','RENEW_IND','SUSCRIPTION_STATUS','TAX_COUNTRY','WINBACK_IND','TAX_STATE','PROMO_CODE','EXPERIENCE_ID','LAST_UPD_BY','SPORT_SEASON_DM_ID','CAMPAIGN_DM_ID','BILL_ZIP_CD','DEVICE_DM_ID']
list_cnv_op=['BATCH_ID','CURRENCY_DM_ID','GOLD_SUBSCRIPTION_ID','SPORT_DM_ID','USER_DM_ID','DEVICE_DM_ID']
list_re_zero=['TAX_AMT','TAX_RATE','TAX_REF_AMT','TOTAL_TP_FEES_AMT']


In [91]:
# function to apply string operations
def str_op(list):
    for i in list:
        df[i]=df[i].apply(lambda x: str(x).strip().upper())
        

In [92]:
#function to apply conversion operations
def cnv_op(list):
    for i in list:
        df[i]=df[i].astype(str)

In [255]:
df.shape

(367844, 45)

In [93]:
# function to replace "?" with 0
def rep_zero(list):
    for i in list:
        df[i].replace(to_replace='^ ?',value=0,regex=True,inplace=True) 

In [219]:
str_op(list_str_op)

In [222]:
cnv_op(list_cnv_op)

In [221]:
rep_zero(list_re_zero)

In [223]:
#to map device_dm_id with respective values and replace others as invalid.New column DEVICE_NM is added to df
df=pd.merge(df,DEVICE_DM_MAP_DF,how='left',on='DEVICE_DM_ID')
df['DEVICE_NM'].fillna('INVALID',inplace=True)
df['DEVICE_NM']=df['DEVICE_NM'].apply(lambda x: str(x).strip().upper())

In [224]:
#to map currency_dm_id with respective values and replace others as invalid.new column Currency_NM is added to df
df=pd.merge(df,CURRENCY_DM_MAP_DF[['CURRENCY_DM_ID','CURRENCY_NM']],how='left',on='CURRENCY_DM_ID')
df['CURRENCY_NM'].fillna('INVALID',inplace=True)
df['CURRENCY_NM']=df['CURRENCY_NM'].apply(lambda x: str(x).strip().upper())

In [225]:
df['CAMPAIGN_DM_ID'].replace(to_replace='?',value='NULL',inplace=True) 
df['CAMPAIGN_DM_ID'].replace(to_replace='-1',value='UNMAPPED',inplace=True) 

In [226]:
df['PROMO_CODE'].replace(to_replace='?',value='NOT APPLICABLE',inplace=True) 

In [227]:
df['PAYMENT_SOURCE_TYPE'].replace(to_replace='?',value='NOT KNOWN',inplace=True) 

In [228]:
df['TAX_COUNTRY'].replace(to_replace='?',value='NOT KNOWN',inplace=True) 

In [229]:
df['TAX_STATE'].replace(to_replace='?',value='NOT KNOWN',inplace=True) 

In [230]:
df['WINBACK_IND'].replace(to_replace='?',value='NEW USER',inplace=True) 

In [231]:
df['BILL_ZIP_CD'].replace(to_replace='?',value='NOT KNOWN',inplace=True) 

In [233]:
del_li=['DEL_IND','EXPERIENCE_ID','LAST_UPD_BY','LAST_UPD_DT_TIME','SUMMARY_DESC']
df.drop(df[del_li],axis =1,inplace=True)

In [252]:
df.to_excel(excel_writer=r'C:\Users\bhavrang\Desktop\bhavya\Analytics\NBCdata.xlsx')

In [242]:
#newdf['DEVICE_NM']=newdf['DEVICE_NM'].astype('category')

In [483]:
df['RENEW_IND'].unique()

array(['N', 'Y'], dtype=object)

In [505]:
new_df=df[['USER_DM_ID','CURRENCY_NM','DEVICE_NM','PAYMENT_SOURCE_TYPE','Sport name','DATA_SOURCE_STATUS','WINBACK_IND','AUTO_RENEW_STATUS','RENEW_IND']].copy()

In [506]:
new_df=pd.get_dummies(new_df, columns=['CURRENCY_NM','DEVICE_NM','PAYMENT_SOURCE_TYPE','Sport name','DATA_SOURCE_STATUS','WINBACK_IND','AUTO_RENEW_STATUS','RENEW_IND'])

In [507]:
#to sum up all the records per user
new_df=new_df.groupby('USER_DM_ID').sum()
#new_df['USER_DM_ID']=new_df.index

In [508]:
#to get the count of the unique subscriptions for a user 
#do it after filtering unique userids else we will get the value repeated
res=df.groupby('USER_DM_ID')['SUBSCRIPTION_ID'].nunique().reset_index(name='UNIQUE_SUB_COUNT')

In [509]:
#res[res['USER_DM_ID']=='100053']

,USER_DM_ID,UNIQUE_SUB_COUNT
47,100053,2


In [510]:
#after taking subscription count only we need to join the dfs for user_dm_id
new_df=pd.merge(new_df,res,how='left',on='USER_DM_ID')
#df['DEVICE_NM'].fillna('INVALID',inplace=True)
#df['DEVICE_NM']=df['DEVICE_NM'].apply(lambda x: str(x).strip().upper())

In [519]:
#to count the total amount charged per user 
chargedf=df[['USER_DM_ID','CHARGE_COUNT','SUBCRIPTION_PRICE','TOTAL_TP_FEES_AMT','TAX_AMT','TAX_REF_AMT','TOTAL_CHARGED_AMT','TOTAL_REFUNDED_AMT','TOTAL_TP_FEES_REFUNDED_AMT']].copy()
chargedf=chargedf.groupby('USER_DM_ID').sum()
new_df=pd.merge(new_df,chargedf,how='left',on='USER_DM_ID')

In [520]:
new_df[new_df['USER_DM_ID']=='100053']

,USER_DM_ID,CURRENCY_NM_EUR,CURRENCY_NM_GBP,CURRENCY_NM_USD,DEVICE_NM_DESKTOP,DEVICE_NM_GAME_CONSOLE,DEVICE_NM_INVALID,DEVICE_NM_MOBILE,DEVICE_NM_OTHER,DEVICE_NM_TABLET,...,TOTAL_REFUNDED_AMT_x,TOTAL_TP_FEES_REFUNDED_AMT_x,CHARGE_COUNT_y,SUBCRIPTION_PRICE_y,TOTAL_TP_FEES_AMT_y,TAX_AMT_y,TAX_REF_AMT_y,TOTAL_CHARGED_AMT_y,TOTAL_REFUNDED_AMT_y,TOTAL_TP_FEES_REFUNDED_AMT_y
51,100053,3,0,0,0,0,3,0,0,0,...,0.00,0.0,5,164.97,207.0,138.63,0,164.97,54.99,0.0
52,100053,3,0,0,0,0,3,0,0,0,...,54.99,0.0,5,164.97,207.0,138.63,0,164.97,54.99,0.0
53,100053,3,0,0,0,0,3,0,0,0,...,0.00,0.0,5,164.97,207.0,138.63,0,164.97,54.99,0.0


In [454]:
#to get the status of a user by verifiying his all subscriptions
DSS=df.groupby(['USER_DM_ID','DATA_SOURCE_STATUS'])['DATA_SOURCE_STATUS'].count().reset_index(name='COUNT')

In [455]:
DSS['USER_STATUS'] = np.where(DSS['DATA_SOURCE_STATUS']=='ACTIVE', 0, 1)

In [456]:
res1=DSS.groupby('USER_DM_ID').sum()

In [457]:
new_df=pd.merge(new_df,res1,how='left',on='USER_DM_ID')
new_df.drop(['COUNT'],axis=1,inplace=True)
new_df['USER_STATUS'].replace(to_replace=0,value='ACTIVE',inplace=True)
new_df['USER_STATUS'].replace(to_replace=1,value='CANCELLED',inplace=True)

In [598]:
new_df[new_df['USER_STATUS']=='CANCELLED'].head()

In [461]:
#total no of records with atleast 1 subscritpion as cancelled.
new_df[new_df['USER_STATUS']=='CANCELLED']['USER_DM_ID'].count()

7046

In [475]:
df['CURRENCY_NM'].unique()

array(['USD', 'EUR', 'GBP'], dtype=object)

In [ ]:
res1=dss.groupby('USER_DM_ID').agg({'USER_DM_ID':'unique'})
res2=dss.groupby('USER_DM_ID').agg({'STATUS':'unique'})

In [ ]:
res_df['USER_DM_ID']=res1['USER_DM_ID']
res_df['STATUS']=res2['STATUS']

In [323]:
chargedf=df[['USER_DM_ID','SUBSCRIPTION_ID','SUBCRIPTION_PRICE','DATA_SOURCE_STATUS']]

In [326]:
df[df['USER_DM_ID']=='39']

,GOLD_SUBSCRIPTION_ID,SUBSCRIPTION_ID,USER_DM_ID,SPORT_DM_ID,Sport name,SPORT_TERM_DM_ID,SUBSCRIPTION_START_DT,DATA_SOURCE_STATUS,SUSCRIPTION_STATUS,CURRENCY_DM_ID,...,TERM_CONV_DT,SPORT_SEASON_DM_ID,TAX_REF_AMT,WINBACK_IND,PAYMENT_SOURCE_TYPE,SUB_ST_DT,SUB_END_DT,NEW_SUMMARY_DESC,DEVICE_NM,CURRENCY_NM
31166,370678,RC4YADSLE79J,39,4,Sport4,24,2018-05-12,ACTIVE,EXISTING,-1,...,?,12,0,A,VISA CARD,"May 18, 2018","May 18,2019",$54.99,INVALID,USD
306543,797468,RCQ9026WCK86,39,29,Sport29,89,2018-12-29,ACTIVE,NEW USER,-1,...,43463.7,27,0,NEW USER,VISA CARD,"Dec 29, 2018","Dec 29, 2019",$89.99,MOBILE,USD


In [324]:
chargedf[chargedf['USER_DM_ID']=='39']


,USER_DM_ID,SUBSCRIPTION_ID,SUBCRIPTION_PRICE,DATA_SOURCE_STATUS
31166,39,RC4YADSLE79J,54.99,ACTIVE
306543,39,RCQ9026WCK86,89.99,ACTIVE


In [523]:
df.dtypes

GOLD_SUBSCRIPTION_ID                   object
SUBSCRIPTION_ID                        object
USER_DM_ID                             object
SPORT_DM_ID                            object
Sport name                             object
SPORT_TERM_DM_ID                        int64
SUBSCRIPTION_START_DT          datetime64[ns]
DATA_SOURCE_STATUS                     object
SUSCRIPTION_STATUS                     object
CURRENCY_DM_ID                         object
SUBCRIPTION_PRICE                     float64
TOTAL_CHARGED_AMT                     float64
TOTAL_TP_FEES_AMT                     float64
CHARGE_COUNT                            int64
TOTAL_REFUNDED_AMT                    float64
TOTAL_TP_FEES_REFUNDED_AMT            float64
LAST_BILLING_DT                datetime64[ns]
NEXT_BILLING_DT                datetime64[ns]
CANCELLATION_DT                datetime64[ns]
USER_ACCESS_EXP_DT             datetime64[ns]
SUBSCRIPTION_TRIAL_END_DT             float64
SUBSCRIPTION_GRCPD_START_DT    dat

In [535]:
date=df.groupby('USER_DM_ID')['SUBSCRIPTION_START_DT']

In [538]:
# Group the data frame by month and item and extract a number of stats from each group
result =df.groupby(['USER_DM_ID']).agg({'SUBSCRIPTION_START_DT':['max']})   # get the first date per group

In [541]:
result.dtypes

SUBSCRIPTION_START_DT  max    datetime64[ns]
dtype: object

In [548]:
datetime_object = datetime.strptime('2018-12-31', '%Y-%m-%d')

In [560]:
result.head()

,SUBSCRIPTION_START_DT,RECENCY
,max,
USER_DM_ID,,
1,2018-08-02,151 days
100,2018-05-13,151 days
10000,2018-05-18,151 days
100001,2018-08-02,151 days
100002,2018-06-25,151 days


In [568]:
nn['ST_DT']=df['SUBSCRIPTION_START_DT'].apply(lambda x:np.datetime64(x))

In [567]:
nn=pd.DataFrame()

In [566]:
nn['st_dt']

0        2018-01-01
1        2018-01-18
2        2018-01-31
3        2018-01-09
4        2018-01-26
5        2018-01-01
6        2018-01-09
7        2018-01-06
8        2018-01-30
9        2018-01-09
10       2018-01-07
11       2018-01-02
12       2018-01-24
13       2018-01-09
14       2018-01-14
15       2018-01-29
16       2018-01-31
17       2018-01-02
18       2018-01-02
19       2018-01-09
20       2018-01-14
21       2018-01-13
22       2018-01-09
23       2018-01-02
24       2018-01-09
25       2018-01-15
26       2018-01-02
27       2018-01-09
28       2018-01-02
29       2018-01-17
            ...    
367814   2018-12-14
367815   2018-05-11
367816   2018-02-18
367817   2018-12-26
367818   2018-06-04
367819   2018-09-11
367820   2018-02-23
367821   2018-12-29
367822   2018-06-02
367823   2018-12-26
367824   2018-12-05
367825   2018-12-16
367826   2018-05-18
367827   2018-12-29
367828   2018-05-13
367829   2018-12-05
367830   2018-01-27
367831   2018-03-17
367832   2018-09-29


In [569]:
nn['RECENCY']= np.datetime64('2018-12-31')-nn['ST_DT']

In [593]:
re=result['SUBSCRIPTION_START_DT']

In [596]:
re.dtypes

max    datetime64[ns]
dtype: object

In [575]:
nn['days']=nn['RECENCY'].apply(lambda x: x.days)

In [589]:
def score(x):
    if(0<=x<=92):
        return 4
    elif(92<x<=184):
        return 3
    elif(184<x<=275):
        return 2
    elif(x>275):
        return 1

In [590]:
nn['SCORE']=nn['days'].apply(lambda x:score(x))

In [592]:
nn['SCORE'].unique()

array([1, 2, 3, 4], dtype=int64)